In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import requests
import folium


In [9]:
df_toronto = pd.read_csv('toranto.csv')
df_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


# create a map for toronto

In [10]:
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'TAMT12GIIIBN2CCTFQ25IMSCOMRB1MGW5VCXKZWSFP0ZMXUE' # your Foursquare ID
CLIENT_SECRET = '5E11NQGULOV211DC0P520HMJPZINBZWTIKIFCO0SBGSBAJYT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Get the neibourhood name 

In [13]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)

Create a map

In [18]:
address_scarb = 'Scarborough,Toronto'
latitude_scarb = 43.773077
longitude_scarb = -79.257774

In [19]:
map_scarb = folium.Map(location=[latitude_scarb, longitude_scarb], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

Get the top 100 venues in the neighborhood 'Steeles West', from Scarborough

In [21]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rouge, Malvern" are 43.811525000000074, -79.19551721399995.


In [22]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cea448c6a6071493fd28880'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 43,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
        'distance

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
5,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
6,Jimmy The Greek,Greek Restaurant,43.775112,-79.257119
7,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
8,DAVIDsTEA,Tea Room,43.776613,-79.258516
9,Scarborough Town Centre,Shopping Mall,43.775237,-79.257363


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [29]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,15,15,15,15,15,15
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",6,6,6,6,6,6
Cedarbrae,3,3,3,3,3,3
"Clairlea, Golden Mile, Oakridge",11,11,11,11,11,11
"Clarks Corners, Sullivan, Tam O'Shanter",12,12,12,12,12,12
"Cliffcrest, Cliffside, Scarborough Village West",10,10,10,10,10,10
"Dorset Park, Scarborough Town Centre, Wexford Heights",3,3,3,3,3,3
"East Birchmount Park, Ionview, Kennedy Park",5,5,5,5,5,5


In [39]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


In [31]:

# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,Auto Garage,Bakery,Bar,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Business Service,Camera Store,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Hong Kong Restaurant,Indian Restaurant,Intersection,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Metro Station,Noodle House,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Restaurant,Sandwich Place,Shanghai Restaurant,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Thai Restaurant,Trail,Train Station,Vietnamese Restaurant
0,"Highland Creek, Rouge Hill, Port Union",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Woburn,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
scarb_onehot.shape

(93, 55)

In [32]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,Auto Garage,Bakery,Bar,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Business Service,Camera Store,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Hong Kong Restaurant,Indian Restaurant,Intersection,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Metro Station,Noodle House,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Restaurant,Sandwich Place,Shanghai Restaurant,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Thai Restaurant,Trail,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.066667,0.0,0.066667,0.0,0.000000,0.000000,0.0,0.0,0.133333,0.000000,0.000000,0.0,0.0,0.066667,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.066667,0.000000,0.0,0.000000,0.000000,0.066667,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.066667,0.00,0.000000,0.000000,0.066667,0.0,0.0,0.066667,0.133333,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.0,0.066667
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.50,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.166667,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.166667,0.0,0.000000,0.000000,0.166667,0.0,0.166667,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.166667,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Cedarbrae,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.333333,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.181818,0.0,0.000000,0.0,0.181818,0.090909,0.0,0.0,0.000000,0.181818,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.181818,0.0,0.0,0.0,0.000000,0.090909,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.083333,0.083333,0.000000,0.0,0.0,0.000000,0.0,0.0,0.083333,0.0,0.000000,0.0,0.083333,0.000000,0.000000,0.0,0.000000,0.083333,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.25,0.166667,0.000000,0.000000,0.0,0.0,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.0,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.0,0.000000,0.0,0.000000,0.1,0.000000,0.000000,0.0,0.0,0.000000,0.100000,0.000000,0.0,0.0,0.000000,0.1,0.2,0.000000,0.1,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.1,0.000000,0.000000,0.0,0.0,0.000000,0.10,0.100000,0.000000,0.000000,0.0,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [46]:
scarb_grouped.shape

(15, 55)

Get top 10 venues

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Hong Kong Restaurant,Department Store,Park,Grocery Store,Pool,Shanghai Restaurant,Vietnamese Restaurant,Bubble Tea Shop
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Sushi Restaurant,Vietnamese Restaurant,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
2,"Birch Cliff, Cliffside West",Park,Gym,General Entertainment,Skating Rink,College Stadium,Gym Pool,Vietnamese Restaurant,Grocery Store,Golf Course,Gift Shop
3,Cedarbrae,Trail,Lounge,Playground,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Intersection,Bus Line,Coffee Shop,Soccer Field,Bus Station,Metro Station,Vietnamese Restaurant,Discount Store,Grocery Store
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pharmacy,Pizza Place,Shopping Mall,Golf Course,Fried Chicken Joint,Coffee Shop,Chinese Restaurant,Hobby Shop,Thai Restaurant,Bus Line
6,"Cliffcrest, Cliffside, Scarborough Village West",Fast Food Restaurant,Pizza Place,Burger Joint,Furniture / Home Store,Liquor Store,Sandwich Place,Coffee Shop,Discount Store,Pharmacy,Convenience Store
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Bakery,Light Rail Station,Gift Shop,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store
8,"East Birchmount Park, Ionview, Kennedy Park",Hobby Shop,Convenience Store,Coffee Shop,Discount Store,Department Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop
9,"Guildwood, Morningside, West Hill",Construction & Landscaping,Park,Gym / Fitness Center,Bar,Bubble Tea Shop,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment


Run k-means to cluster the neighborhoods into 5 clusters

In [43]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 2, 0, 4, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues.

In [76]:
# add clustering labels

scarb_merged = scarb_data.iloc[0:15,:]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarb_merged.head() # check the last columns!




,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,1.0,Bar,Vietnamese Restaurant,Convenience Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256,0.0,Construction & Landscaping,Park,Gym / Fitness Center,Bar,Bubble Tea Shop,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment
3,M1G,Scarborough,Woburn,43.768359,-79.217590,0.0,Korean Restaurant,Business Service,Coffee Shop,Park,Vietnamese Restaurant,Department Store,Gym,Grocery Store,Golf Course,Gift Shop
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,4.0,Trail,Lounge,Playground,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store


In [79]:
scarb_merged["Cluster Labels"] = scarb_merged["Cluster Labels"].fillna(0.0).astype(int)

In [80]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighborhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [81]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Scarborough,0,Construction & Landscaping,Park,Gym / Fitness Center,Bar,Bubble Tea Shop,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment
3,Scarborough,0,Korean Restaurant,Business Service,Coffee Shop,Park,Vietnamese Restaurant,Department Store,Gym,Grocery Store,Golf Course,Gift Shop
5,Scarborough,0,Train Station,Indian Restaurant,Grocery Store,Restaurant,Convenience Store,Gym,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
6,Scarborough,0,Hobby Shop,Convenience Store,Coffee Shop,Discount Store,Department Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop
7,Scarborough,0,Bakery,Intersection,Bus Line,Coffee Shop,Soccer Field,Bus Station,Metro Station,Vietnamese Restaurant,Discount Store,Grocery Store
8,Scarborough,0,Fast Food Restaurant,Pizza Place,Burger Joint,Furniture / Home Store,Liquor Store,Sandwich Place,Coffee Shop,Discount Store,Pharmacy,Convenience Store
9,Scarborough,0,Park,Gym,General Entertainment,Skating Rink,College Stadium,Gym Pool,Vietnamese Restaurant,Grocery Store,Golf Course,Gift Shop
10,Scarborough,0,Bakery,Light Rail Station,Gift Shop,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store
12,Scarborough,0,Chinese Restaurant,Shopping Mall,Hong Kong Restaurant,Department Store,Park,Grocery Store,Pool,Shanghai Restaurant,Vietnamese Restaurant,Bubble Tea Shop


In [82]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,Bar,Vietnamese Restaurant,Convenience Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store


In [83]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,2,Pharmacy,Sushi Restaurant,Vietnamese Restaurant,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint


In [84]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,3,Auto Garage,Intersection,Convenience Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint


In [85]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,4,Trail,Lounge,Playground,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
